# Estimación probabilística de la dosis de exposición inhalatoria


## Estimación de la concentración de la sustancia de interés en la fase vapor

- *pv* hace referencia a la presión de vapor, en Pascales (Pa), a la temperatura estándar o normal, de la sustancia de interés
- *mw* se refiere al peso molecular de la sustancia de interés, en (mg/mol)
- *mean* corresponde al valor de la media de la concentración de la sustancia de interés en la solución a considerar.
- *stdev* es la desviación estándar de la concentración de la sustancia de interés en la solución utilizada.
- *CV* es el coeficiente de variación es la desviación estándar relativa  de la concentración de la sustancia de interés en la solución utilizada.
- *dosis* es la cantidad de mg de ingrediente activo por hectarea (mg *i.a*/Ha) a ser utilizada durante la aplicación del producto químico de uso agrícola de que se trate.
- *volumen de aspersión* es el volumen por hectarea (L/Ha) de solución o dispersión de la sustancia de interés a ser utilizada.


In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from scipy.stats import lognorm
from scipy.stats import iqr
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
%matplotlib inline

dtcnc = []

# Initialize the values of pv, mw, and cv
global pv, mw, cv, masc1, masc2
pv = 8.7e-06  # Initial value of pressure (Pa)
mw = 301.39  # Initial value of molecular weight (g/mol)
cv = 1.607   # Initial value of Coefficient of Variation (CV)
masc1 = False # Initial value for checkbox_masc1
masc2 = False # Initial value for checkbox_masc2

# Define the function to be plotted
def plot_function(pv, mw, cv):
    global dtcnc
    mean = ((pv / 101325) * mw) / (0.082 * (20 + 273))  # Calculate vapconc based on pv and mw
    mean *= 1.0e6  # Convert to mg/m^3
    
    stdev = cv * mean

    phi = (stdev ** 2 + mean ** 2) ** 0.5
    mu = np.log(mean ** 2 / phi)
    sigma = (np.log(phi ** 2 / mean ** 2)) ** 0.5
    
    dtcnc = np.random.lognormal(mu, sigma, 5000)
    
    plt.figure(figsize=(10, 8))
    plt.hist(dtcnc, bins=30, density=True, alpha=0.5, color="b")
    
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 1000)
    p = lognorm.pdf(x=x, scale=mean, s=sigma)
    plt.plot(x, p, 'k', linewidth=2)
    
    plt.ylabel('Densidad de probabilidad')
    plt.xlabel('mg/m$^{3}$', y=0.05)
    
    plt.title(f'Distribución de la concentración en fase vapor (pv={pv} Pa, mw={mw} g/mol, CV={cv})')
    
    plt.savefig('log_normal_vap.png', bbox_inches='tight', dpi=300)
#     plt.show()
    
    return dtcnc

# Define a custom format function for the slider
def format_engineering_notation(value):
    return f"{value:.2e}"

# Define the slider widgets for pv, mw, and cv
slider_pv = widgets.FloatSlider(min=1e-07, max=1e-05, step=1e-08, value=pv, description='Pv (Pa)',
                                readout_format='.2e')  # Use the custom format
slider_mw = widgets.FloatSlider(min=100, max=500, step=1, value=mw, description='mw (g/mol)')
slider_cv = widgets.FloatSlider(min=0, max=5, step=0.01, value=cv, description='CV')

def observe_slider_pv(change):
    global pv
    pv = 10 ** change["new"]

def observe_slider_mw(change):
    global mw
    mw = change["new"]

def observe_slider_cv(change):
    global cv
    cv = change["new"]

slider_pv.observe(observe_slider_pv, names="value")
slider_mw.observe(observe_slider_mw, names="value")
slider_cv.observe(observe_slider_cv, names="value")

# Create the interactive plot
interactive_plot = widgets.interactive(plot_function, pv=slider_pv, mw=slider_mw, cv=slider_cv)
display(interactive_plot)

dtcnc = interactive_plot.result  

interactive(children=(FloatSlider(value=8.7e-06, description='Pv (Pa)', max=1e-05, min=1e-07, readout_format='…

In [2]:
inhalkde = pd.read_csv('tasainhal.csv').loc[:, '0'].values

In [3]:
pesokde = pd.read_csv('pesokde.csv').loc[:, '0'].values

In [4]:
def hstgrm(xpinhal, bins, xlim):
    plt.figure(figsize=(10,8))
    plt.xlim(xlim)
    plt.hist(xpinhal, density=True, bins=bins)  # Fixed the variable name here
    plt.ylabel('Densidad de probabilidad')
    plt.xlabel('mg/kg·día')
    plt.axvline(x=np.percentile(xpinhal, 50), linewidth=1, color='r', linestyle=':')
    plt.axvline(x=np.percentile(xpinhal, 2.5), linewidth=1, color='r', linestyle='--')
    plt.axvline(x=np.percentile(xpinhal, 97.5), linewidth=1, color='r', linestyle='dashed')
    plt.text((np.percentile(xpinhal, 97.5) + 0.00005 * iqr(xpinhal)), len(xpinhal) / sum(xpinhal),
             'P97.5 = ' + np.format_float_scientific(np.percentile(xpinhal, 97.5), precision=2), rotation=90, color='red',
             verticalalignment='center')
    plt.savefig('exp_inhalatoria.png', bbox_inches='tight', dpi=300)
    plt.show()

def montecarlo(dtcnc, mw, N, masc1, masc2):
    random_vapconc = np.random.choice(dtcnc, size=N)
    random_inhal = np.random.choice(inhalkde, size=N)
    random_peso = np.random.choice(pesokde, size=N)
    if masc1:
        masc = 0.25
        expinhal = ((random_vapconc * random_inhal * masc) / random_peso)
    elif masc2:
        masc = 0.10
        expinhal = ((random_vapconc * random_inhal * masc) / random_peso)
    else:
        expinhal = ((random_vapconc * random_inhal) / random_peso)
    return expinhal

def hstgrm_wrapper(dtcnc, mw, bins, masc1, masc2):
    xpinhal = montecarlo(dtcnc, mw, 10000, masc1, masc2)
    xlim = (0, np.percentile(xpinhal, 97.5) + iqr(xpinhal) + (0.01 * iqr(xpinhal)))
    plt.clf()
    plt.figure(figsize=(10, 8))
    hstgrm(xpinhal, bins, xlim)
    plt.figure(figsize=(10, 8))
    plt.boxplot(np.array(xpinhal)[np.array(xpinhal < np.percentile(xpinhal, 97.5) + iqr(xpinhal))], vert=False,
                patch_artist=True)
    plt.xlabel('mg/kg·día')
    plt.yticks([1], [''])
    plt.axvline(x=np.quantile(xpinhal, 0.975), linewidth=1, color='r', linestyle='--')
    plt.text((np.percentile(xpinhal, 97.5) + (np.percentile(xpinhal, 97.5) * 1.0e-05)), 1.125,
             'P97.5 = ' + np.format_float_scientific(np.percentile(xpinhal, 97.5), precision=2), color='red', rotation=90)
    plt.xlim(xlim)
    plt.savefig('boxplot_inhalatoria.png', bbox_inches='tight', dpi=300)
    plt.show()

# Define the checkboxes with observe functions for mutual exclusivity
checkbox_masc1 = widgets.Checkbox(value=False, description='uso de máscara facial FP1')
checkbox_masc2 = widgets.Checkbox(value=False, description='uso de máscara facial FFP2')

def observe_masc1(change):
    global masc1, masc2
    if change["new"]:
        masc2 = False
        checkbox_masc2.value = False

def observe_masc2(change):
    global masc1, masc2
    if change["new"]:
        masc1 = False
        checkbox_masc1.value = False

checkbox_masc1.observe(observe_masc1, names="value")
checkbox_masc2.observe(observe_masc2, names="value")

slider_mean = widgets.FloatSlider(min=0, max=10, step=0.01, value=0.07)
slider_bins = widgets.IntSlider(min=1, max=10000, step=1, value=1000)
slider_pv = widgets.FloatSlider(min=1e-07, max=1e-05, step=1e-08, value=pv, description='Pv (Pa)',
                                readout_format='.2e')  # Use the custom format
slider_mw = widgets.FloatSlider(min=100, max=500, step=1, value=mw, description='mw (g/mol)')
slider_cv = widgets.FloatSlider(min=0, max=5, step=0.01, value=cv, description='CV')

def observe_slider_pv(change):
    global pv
    pv = 10 ** change["new"]

def observe_slider_mw(change):
    global mw
    mw = change["new"]

def observe_slider_cv(change):
    global cv
    cv = change["new"]

def interactive_plot_func(pv, mw, cv, bins, masc1, masc2):
    # Call your plot_function here with pv, mw, and cv
    plot_function(pv, mw, cv)
    # Call hstgrm_wrapper with the updated parameters
    hstgrm_wrapper(dtcnc, mw, bins, masc1, masc2)
    
slider_pv.observe(observe_slider_pv, names="value")
slider_mw.observe(observe_slider_mw, names="value")
slider_cv.observe(observe_slider_cv, names="value")


interactive_plot = widgets.interactive(interactive_plot_func,
                                       pv=slider_pv,
                                       mw=slider_mw,
                                       cv=slider_cv,
                                       bins=slider_bins,
                                       masc1=checkbox_masc1,
                                       masc2=checkbox_masc2)  # Pass both masc1 and masc2

display(interactive_plot)

interactive(children=(FloatSlider(value=8.7e-06, description='Pv (Pa)', max=1e-05, min=1e-07, readout_format='…